# 基于缺失值与异常值的分析对Lending Club数据进行预处理

In [23]:
#coding=utf-8
import pandas as pd
import numpy as np

In [24]:
### 读取原始数据
data = pd.read_csv('loan.csv')
print(data.head(10))


D:\pro\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0 NaN        NaN      14000        14000          14000.0   36 months   
1 NaN        NaN      35000        35000          34725.0   36 months   
2 NaN        NaN      21000        21000          21000.0   60 months   
3 NaN        NaN       6400         6400           6400.0   36 months   
4 NaN        NaN       9500         9500           9500.0   36 months   
5 NaN        NaN      30000        30000          30000.0   36 months   
6 NaN        NaN      12000        12000          12000.0   36 months   
7 NaN        NaN       7000         7000           7000.0   36 months   
8 NaN        NaN      28000        28000          28000.0   36 months   
9 NaN        NaN      20500        20500          20500.0   36 months   

   int_rate  installment grade sub_grade  ... hardship_payoff_balance_amount  \
0     13.49       475.03     C        C1  ...                            NaN   
1     17.27      1252.56     D      

In [25]:
### 查看数据集缺失情况
missDf = data.isnull().sum().sort_values(ascending=False).reset_index()
missDf.columns = ['feature', 'miss_num']
# 确实比值
missDf['miss_percentage'] = missDf['miss_num']/len(data)

print(missDf.head(10))

                                      feature  miss_num  miss_percentage
0                                          id    400000         1.000000
1                                   member_id    400000         1.000000
2                                         url    400000         1.000000
3  orig_projected_additional_accrued_interest    399275         0.998188
4                         hardship_start_date    399109         0.997772
5                           hardship_end_date    399109         0.997772
6                     payment_plan_start_date    399109         0.997772
7                             hardship_length    399109         0.997772
8                                hardship_dpd    399109         0.997772
9                        hardship_loan_status    399109         0.997772


In [34]:
### 统计缺失值
print('共有缺失列数：', missDf[missDf['miss_percentage']>0].shape[0])
print('缺失数大于30%列数：', missDf[missDf['miss_percentage']>0.3].shape[0])

共有缺失列数： 112
缺失数大于30%列数： 58


In [36]:
### 处理方法1--直接删除
# 适应场景：缺失值比例过大的特征可以考虑直接整列删除。
# 阈值设 0.3
thr = (1-0.3) * data.shape[0]
data = data.dropna(thresh=thr, axis=1)
print(data.head(10))


   loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  installment  \
0      14000        14000          14000.0   36 months     13.49       475.03   
1      35000        35000          34725.0   36 months     17.27      1252.56   
2      21000        21000          21000.0   60 months     19.42       549.62   
3       6400         6400           6400.0   36 months     10.65       208.47   
4       9500         9500           9500.0   36 months     11.71       314.23   
5      30000        30000          30000.0   36 months     17.27      1073.62   
6      12000        12000          12000.0   36 months      7.90       375.49   
7       7000         7000           7000.0   36 months     12.69       234.82   
8      28000        28000          28000.0   36 months     14.27       960.65   
9      20500        20500          20500.0   36 months      7.90       641.46   

  grade sub_grade                           emp_title emp_length  ...  \
0     C        C1            Eric J

In [47]:
### 缺失特征过多的样本可以考虑直接整行删除
data['miss_rowNums'] = data.apply(lambda x:x.isnull().sum(), axis=1)
print(data['miss_rowNums'])
print(data['miss_rowNums'].value_counts())

0         34
1         34
2         34
3         34
4         34
          ..
399995     0
399996     0
399997     0
399998     2
399999     1
Name: miss_rowNums, Length: 395625, dtype: int64
0     273488
1      51951
34     41348
2      19353
3       4755
4       3494
5        721
6        365
7        120
8         26
9          3
10         1
Name: miss_rowNums, dtype: int64


In [49]:
### 删掉缺失值大于35列的样本
data = data[data['miss_rowNums']<35]
data.drop(['miss_rowNums'], axis=1, inplace=True)
print(data)

        loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  \
0           14000        14000          14000.0   36 months     13.49   
1           35000        35000          34725.0   36 months     17.27   
2           21000        21000          21000.0   60 months     19.42   
3            6400         6400           6400.0   36 months     10.65   
4            9500         9500           9500.0   36 months     11.71   
...           ...          ...              ...         ...       ...   
399995      12000        12000          12000.0   60 months     14.08   
399996      12000        12000          12000.0   60 months     25.82   
399997      10000        10000          10000.0   36 months     11.99   
399998      12000        12000          12000.0   60 months     21.45   
399999      16550        16550          16550.0   60 months     21.45   

        installment grade sub_grade                 emp_title emp_length  ...  \
0            475.03     C        C1  Eric 

In [55]:
### 处理方法2--固定值填充
# 特征emp_length中的缺失值可以尝试用特殊字符表示，代表独立的一类特征值。
print(data['emp_length'].value_counts())
# 将emp_length特征列中的缺失值替换为特殊字符'Unknown'
data.loc[data['emp_length']=='n/a', 'emp_length'] = 'Unknown'
print(data['emp_length'].value_counts())


10+ years    127152
2 years       35782
< 1 year      35591
3 years       31895
1 year        25612
4 years       24503
5 years       23573
7 years       19366
6 years       19364
8 years       16677
9 years       14369
Name: emp_length, dtype: int64
10+ years    127152
2 years       35782
< 1 year      35591
3 years       31895
1 year        25612
4 years       24503
5 years       23573
7 years       19366
6 years       19364
8 years       16677
9 years       14369
Name: emp_length, dtype: int64


In [60]:
### 处理方法3--统计值填充
### dti属性是数值特征，可以使用均值或中位数进行填充
print('填充前：', data['dti'].isnull().sum())
print('填充前：', data['dti'].describe())
data['dti'].fillna(data['dti'].median(), inplace=True)
print('填充后：', data['dti'].describe())
print('填充后：', data['dti'].isnull().sum())

填充前： 222
填充前： count    395403.000000
mean         18.002763
std          12.966729
min           0.000000
25%          11.390000
50%          17.150000
75%          23.480000
max         999.000000
Name: dti, dtype: float64
填充后： count    395625.000000
mean         18.002284
std          12.963106
min           0.000000
25%          11.390000
50%          17.150000
75%          23.470000
max         999.000000
Name: dti, dtype: float64
填充后： 0


In [61]:
# 处理方法4--建模填充
### 以revol_util(信用账户的使用率)特征为例，导入sklearn的随机森林算法预测缺失值
from sklearn.ensemble import RandomForestRegressor    
from sklearn.metrics import r2_score
rfDf = data.copy()
# 用revol_util特征值非空的样本构建训练集，revol_util特征值缺失的样本构建测试集
rfDf_train = rfDf.loc[rfDf['revol_util'].notnull()]
rfDf_test = rfDf.loc[rfDf['revol_util'].isnull()]

col = ['loan_amnt', 'int_rate', 'installment', 'revol_bal', 'collection_recovery_fee']    # 原始数据集中的无缺失数值特征
# 划分训练数据和标签（label）
X = rfDf_train[col]
y = rfDf_train['revol_util']
# 训练过程
rf = RandomForestRegressor(n_estimators=100,n_jobs=-1)    # 这里重在理解过程，因此仅简单选取部分参数
rf.fit(X, y)
# 预测过程
pred = rf.predict(rfDf_test[col])
rfDf.loc[(rfDf['revol_util'].isnull()), 'revol_util'] = pred    # 填补缺失值
print("此时的revol_util特征统计指标:\n")
print(rfDf['revol_util'].describe()) 

此时的revol_util特征统计指标:

count    395625.000000
mean         51.670616
std          24.870029
min           0.000000
25%          33.000000
50%          52.100000
75%          70.900000
max         892.300000
Name: revol_util, dtype: float64
